In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv("Dataset_Kinerja_Kepolisian.csv")

In [4]:
data.head()

,Unnamed: 0,id,created_at,text
0,0,1587062096306372608,2022-10-31 12:41:04+00:00,Puan menuntut Polri transparan dalam pengusuta...
1,1,1587055787905814528,2022-10-31 12:16:00+00:00,Kalo ada yg nanya knp sih g' ada film/series l...
2,2,1587054014721454080,2022-10-31 12:08:57+00:00,@siapkawalpolri Betul memang tugas polri selal...
3,3,1587042748162793473,2022-10-31 11:24:11+00:00,Kenaikan pangkat Pengabdian bukan merupakan ke...
4,4,1587042697302642688,2022-10-31 11:23:59+00:00,@Res1Tulungagung Kenaikan pangkat Pengabdian b...


# CASE FOLDING

In [5]:
data['text'] = data['text'].str.lower()
print('Case Folding : \n')
print(data['text'].head())

Case Folding : 

0    puan menuntut polri transparan dalam pengusuta...
1    kalo ada yg nanya knp sih g' ada film/series l...
2    @siapkawalpolri betul memang tugas polri selal...
3    kenaikan pangkat pengabdian bukan merupakan ke...
4    @res1tulungagung kenaikan pangkat pengabdian b...
Name: text, dtype: object


# TOKENIZING

In [6]:
import string 
import re 


from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist


def remove_tweet_special(text):
    # MENGHAPUS tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # MENGHAPUS non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # MENGHAPUS mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # MENGHAPUS incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data['text'] = data['text'].apply(remove_tweet_special)

#MENGHAPUS number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data['text'] = data['text'].apply(remove_number)

#MENGHAPUS punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data['text'] = data['text'].apply(remove_punctuation)

#MENGHAPUS whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data['text'] = data['text'].apply(remove_whitespace_LT)

#MENGHAPUS multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data['text'] = data['text'].apply(remove_whitespace_multiple)

# MENGHAPUS single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data['text'] = data['text'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data['text_tokens'] = data['text'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data['text_tokens'].head())

Tokenizing Result : 

0    [puan, menuntut, polri, transparan, dalam, pen...
1    [kalo, ada, yg, nanya, knp, sih, ada, filmseri...
2    [betul, memang, tugas, polri, selalu, mengayom...
3    [kenaikan, pangkat, pengabdian, bukan, merupak...
4    [kenaikan, pangkat, pengabdian, bukan, merupak...
Name: text_tokens, dtype: object


In [7]:
def freqDist_wrapper(text):
    return FreqDist(text)

data['text_tokens_fdist'] = data['text_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(data['text_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(polri, 3), (dia, 2), (puan, 1), (menuntut, 1...
1    [(yg, 4), (ada, 2), (kita, 2), (ini, 2), (kalo...
2    [(polri, 2), (betul, 1), (memang, 1), (tugas, ...
3    [(merupakan, 2), (kenaikan, 1), (pangkat, 1), ...
4    [(merupakan, 2), (kenaikan, 1), (pangkat, 1), ...
Name: text_tokens_fdist, dtype: object


# STOPWORDS

In [8]:
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')

list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
                      
# convert list to dictionary
list_stopwords = set(list_stopwords)

def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data['text_tokens_SW'] = data['text_tokens'].apply(stopwords_removal) 


print(data['text_tokens_SW'].head())

0    [puan, menuntut, polri, transparan, pengusutan...
1    [nanya, knp, filmseries, lokal, ceritanya, men...
2    [tugas, polri, mengayomi, melindungi, masyarak...
3    [kenaikan, pangkat, pengabdian, keharusan, has...
4    [kenaikan, pangkat, pengabdian, keharusan, has...
Name: text_tokens_SW, dtype: object


# Stemmed

In [9]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data['text_tokens_SW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data['text_tokens_stemmed'] = data['text_tokens_SW'].swifter.apply(get_stemmed_term)
print(data['text_tokens_stemmed'])

651
------------------------
puan : puan
menuntut : tuntut
polri : polri
transparan : transparan
pengusutan : usut
ditangani : tangan
meningkatkan : tingkat
perlindungan : lindung
masyarakat : masyarakat
dpr : dpr
pengawasan : awas
kinerja : kerja
presiden : presiden
perempuan : perempuan
nanya : nanya
knp : knp
filmseries : filmseries
lokal : lokal
ceritanya : cerita
mengkritik : kritik
polisi : polisi
mingil : mingil
saranin : saranin
nonton : nonton
seriesfilm : seriesfilm
india : india
kultur : kultur
deket : deket
emank : emank
berani : berani
banget : banget
tema : tema
sejenis : jenis
salah : salah
satunya : satu
delhi : delhi
crime : crime
season : season
tugas : tugas
mengayomi : ayom
melindungi : lindung
semoga : moga
kedepannya : depan
meningkat : tingkat
kenaikan : naik
pangkat : pangkat
pengabdian : abdi
keharusan : harus
hasil : hasil
penilaian : nilai
prestasi : prestasi
personil : personil
sungguhsungguh : sungguhsungguh
mengabdikan : abdi
insitusi : insitusi
setuju : t

george : george
irjen : irjen
suharyono : suharyono
mengapresiasi : apresiasi
tragedi : tragedi
kanjuruhan : kanjuruhan
ironi : ironi
memahaminya : paham
sekedar : dar
pencitraan : citra
perubahan : ubah
zaman : zaman
canggih : canggih
salahnya : salah
teknik : teknik
efisiensi : efisiensi
program : program
dpat : dpat
berjalan : jalan
lancar : lancar
etilang : etilang
berguna : guna
jaman : jaman
digital : digital
praktis : praktis
mengefesiensikan : mengefesiensikan
imigrasi : imigrasi
pelaku : laku
gagal : gagal
bayar : bayar
wanaartha : wanaartha
diperbolehkan : boleh
kabur : kabur
kejadian : jadi
dicekal : cekal
dibekukan : beku
rekening : rekening
ditahan : tahan
transaksi : transaksi
pendamping : damping
berubah : ubah
membantu : bantu
teruslah : terus
informasi : informasi
kegiatan : giat
penanggung : tanggung
ikhwan : ikhwan
sanusi : sanusi
refleksi : refleksi
membawa : bawa
poin : poin
seruan : seru
keresahan : resah
demokrasi : demokrasi
direpresif : represif
revolusi : revo

Pandas Apply:   0%|          | 0/200 [00:00<?, ?it/s]

0      [puan, tuntut, polri, transparan, usut, tangan...
1      [nanya, knp, filmseries, lokal, cerita, kritik...
2      [tugas, polri, ayom, lindung, masyarakat, moga...
3      [naik, pangkat, abdi, harus, hasil, nilai, pre...
4      [naik, pangkat, abdi, harus, hasil, nilai, pre...
                             ...                        
195    [ri, polri, penggrebekan, makan, kosmetik, akt...
196    [pemuda, jawa, sedia, deklarasi, dukung, kerja...
197    [jmhasil, kerja, generasi, cinta, bangsa, pemu...
198    [percaya, polisi, jujur, moga, kerja, polisi, ...
199    [ragu, polri, lindung, masyarakat, masyarakat,...
Name: text_tokens_stemmed, Length: 200, dtype: object


In [10]:
data.to_csv("Text_Preprocessing_Kinerja.csv")

In [11]:
data.to_excel("Text_Preprocessing_Kinerja.xlsx")